# Sprawozdanie laboratorium pierwsze
Witold Jagiełło-Królikowski
s21165

Adrian Bloch

### Zadanie polega na wykorzystaniu AI do rozponawanie gestów ręki.
#### aby nie odkrywać na nowo koła wykorzystaliśmy już nauczoną sztuczną inteligencje od firmy Google


### Import niezbednych bibliotek
## Jedną z nowo poznanych jest mediapipe dzięki któremu możemy w realnym czasie odczytywać dane uzyskane od cv2 czyli między innymi z kamerki

In [9]:
import cv2
import numpy as np
import mediapipe as mp

from tensorflow.keras.models import load_model

### Inicjalizacja mediapipe
Mp.solution.hands jest odpowiedzialny za rozpoznawanie rąk, konfigurujemy model aby wykrywał jednął rękę i używamy Mp.solutions.drawing_utils aby rysoawł punkty na ręce dzięki którymi będziemy rozpoznawać w jakiej pozycji znajduję się wykryta ręka


In [10]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

## Inicjalizacja Tensorflow
### Używając funkcji load_model ładujemy wytrenowany model TensorFlow, w pliku gesture_names sa zawarte obsługiwane przez nas gesty rąk które będziemy wykrywać, należą do nich między innymi rock, stop, znak pokoju, okay, oraz kciuk w górę, jest ich łącznie 10.

In [11]:
# Load the gesture recognizer model
model = load_model('mp_hand_gesture')

# Load class names
f = open('gesture.names', 'r')
classNames = f.read().split('\n')
f.close()
print(classNames)

['okay', 'peace', 'thumbs up', 'thumbs down', 'call me', 'stop', 'rock', 'live long', 'fist', 'smile']


## Odczytywanie klatek z kamerki :)

In [12]:
# Inicjalizujemy cv2 aby używać kamerki
cap = cv2.VideoCapture(0)

while True:
    # Czytanie po klatce
    _, frame = cap.read()

    x, y, c = frame.shape

    # Obrót obrazu oraz zmiana na format RGB
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    result = hands.process(framergb)



    className = ''

    # jeżeli wykryjemy rękę zaczynamy naszą magie, przechodzimi przez wszystkie wykryte pkt i zapisujemy ich położenie w liście landmarks
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                lmx = int(lm.x * x)
                lmy = int(lm.y * y)

                landmarks.append([lmx, lmy])

            # Nanosimy pkt na uzyskane klatki
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

            # AI stara się dopasować gest
            prediction = model.predict([landmarks])
            # print(prediction)
            classID = np.argmax(prediction)
            className = classNames[classID]

    # Pokazujemy na kamerce jaki gest udało nam się wykryć
    cv2.putText(frame, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0,0,255), 2, cv2.LINE_AA)

    # Kamereczko, powiedz przecie, kto jest najpiekniejszy w świecie?
    cv2.imshow("Output", frame)

    if cv2.waitKey(1) == ord('q'):
        break




1/1 [==============================] - 0s 39ms/step


KeyboardInterrupt: 

In [ ]:
# Na koniec odłączamy się od kamerki i niszczymy aktywne okna
cap.release()

cv2.destroyAllWindows()